#### Učitavanje potrebnih biblioteka za rad

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor,BaggingRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error,r2_score
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
import warnings
import numpy as np
from matplotlib import pyplot as plt

In [2]:
warnings.filterwarnings('ignore')

#### Učitavanje podataka i njihova priprema za modelovanje

In [3]:
final_player_stats=pd.read_csv("./organized_data/Konacni_podaci.csv")
team_name_by_id=pd.read_csv("./organized_data/Timovi_i_id_tima.csv")
granice_poeni=pd.read_csv("./data/granice_igraci.csv")
granice_poeni

,player,points_pros,date,player_team
0,William James Nunnally,13.5,2022-11-24,PAR
1,Zach Leday,12.5,2022-11-24,PAR
2,Alec Peters,13.5,2022-11-24,OLY
3,Isaiah Canaan,11.5,2022-11-24,OLY
4,Thomas Walkup,8.5,2022-11-24,OLY
...,...,...,...,...
2231,Joel Bolomboy,9.5,2024-03-01,RED
2232,Yago Santos Dos,12.5,2024-03-01,RED
2233,Rokas Giedraitis,10.5,2024-03-01,RED
2234,Nemanja Nedovic,13.5,2024-03-01,RED


In [4]:
def get_team_name(team_id):
    return team_name_by_id[team_name_by_id['team_id'] == team_id]['team_name'].values[0]

In [5]:
granice_poeni['player_team_name']=granice_poeni['player_team'].apply(lambda x: get_team_name(x))
granice_poeni.drop(columns=['player_team'],axis=1,inplace=True)
granice_poeni.rename({'player_team_name':'player_team'},inplace=True,axis=1)
granice_poeni

,player,points_pros,date,player_team
0,William James Nunnally,13.5,2022-11-24,PARTIZAN MOZZART BET BELGRADE
1,Zach Leday,12.5,2022-11-24,PARTIZAN MOZZART BET BELGRADE
2,Alec Peters,13.5,2022-11-24,OLYMPIACOS PIRAEUS
3,Isaiah Canaan,11.5,2022-11-24,OLYMPIACOS PIRAEUS
4,Thomas Walkup,8.5,2022-11-24,OLYMPIACOS PIRAEUS
...,...,...,...,...
2231,Joel Bolomboy,9.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE
2232,Yago Santos Dos,12.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE
2233,Rokas Giedraitis,10.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE
2234,Nemanja Nedovic,13.5,2024-03-01,CRVENA ZVEZDA MTS BELGRADE


In [6]:
final_player_stats=pd.merge(final_player_stats,granice_poeni,how='left',on=['player','date','player_team'])
final_player_stats

,player,player_team,opposing_team,date,game_id,season,home_court,Avg_Season_minutes,Avg_Season_points,Avg_Season_two_points_made,...,player_team_ranking,h_off_rating,a_off_rating,opp_def_rating,opp_avg_rec_points,team_avg_scored_points,player_team_win_pct,opposing_team_win_pct,points,points_pros
0,Timothe Luwawu-Cabarrot,LDLC ASVEL VILLEURBANNE,AX ARMANI EXCHANGE OLIMPIA MILAN,2024-03-01,E2023_243,2023,1,29.135714,13.523810,3.571429,...,1,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407,7,10.5
1,Nando Colo De,LDLC ASVEL VILLEURBANNE,AX ARMANI EXCHANGE OLIMPIA MILAN,2024-03-01,E2023_243,2023,1,20.897436,12.153846,1.769231,...,2,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407,13,NaN
2,Joffrey Lauvergne,LDLC ASVEL VILLEURBANNE,AX ARMANI EXCHANGE OLIMPIA MILAN,2024-03-01,E2023_243,2023,1,26.100000,12.083333,4.250000,...,3,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407,17,11.5
3,Paris Lee,LDLC ASVEL VILLEURBANNE,AX ARMANI EXCHANGE OLIMPIA MILAN,2024-03-01,E2023_243,2023,1,28.957051,12.076923,2.076923,...,4,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407,5,11.5
4,Mike Scott,LDLC ASVEL VILLEURBANNE,AX ARMANI EXCHANGE OLIMPIA MILAN,2024-03-01,E2023_243,2023,1,22.953205,11.076923,2.923077,...,5,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407,12,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24503,Brock Motum,ANADOLU EFES ISTANBUL,FC BAYERN MUNICH,2019-01-03,E2018_121,2018,1,12.859524,8.428571,2.142857,...,6,1.070937,1.020805,1.010547,80.800000,86.214286,0.533333,0.500000,2,NaN
24504,Bryant Dunston,ANADOLU EFES ISTANBUL,FC BAYERN MUNICH,2019-01-03,E2018_121,2018,1,24.921429,7.857143,3.142857,...,7,1.070937,1.020805,1.010547,80.800000,86.214286,0.533333,0.500000,16,NaN
24505,James Anderson,ANADOLU EFES ISTANBUL,FC BAYERN MUNICH,2019-01-03,E2018_121,2018,1,16.352381,5.714286,0.857143,...,8,1.070937,1.020805,1.010547,80.800000,86.214286,0.533333,0.500000,16,NaN
24506,Dogus Balbay,ANADOLU EFES ISTANBUL,FC BAYERN MUNICH,2019-01-03,E2018_121,2018,1,10.650000,2.500000,0.714286,...,9,1.070937,1.020805,1.010547,80.800000,86.214286,0.533333,0.500000,0,NaN


#### Izdvajanje atributa potrebnih za modelovanje

In [7]:
X=final_player_stats.drop(columns=['game_id','player','player_team','opposing_team','date','season','points','points_pros'])
y=final_player_stats['points']

In [8]:
X

,home_court,Avg_Season_minutes,Avg_Season_points,Avg_Season_two_points_made,Avg_Season_two_points_attempted,Avg_Season_three_points_made,Avg_Season_three_points_attempted,Avg_Season_free_throws_made,Avg_Season_free_throws_attempted,Avg_Season_offensive_rebounds,...,Last_2_Avg_ts%,avg_points_position,player_team_ranking,h_off_rating,a_off_rating,opp_def_rating,opp_avg_rec_points,team_avg_scored_points,player_team_win_pct,opposing_team_win_pct
0,1,29.135714,13.523810,3.571429,6.857143,1.190476,3.523810,2.809524,3.428571,0.476190,...,0.500000,24.307692,1,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407
1,1,20.897436,12.153846,1.769231,3.615385,1.769231,4.538462,3.307692,3.769231,0.076923,...,0.710388,31.423077,2,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407
2,1,26.100000,12.083333,4.250000,8.041667,0.625000,1.833333,1.708333,2.333333,1.791667,...,0.677296,20.923077,3,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407
3,1,28.957051,12.076923,2.076923,4.615385,1.615385,4.846154,3.076923,3.307692,0.307692,...,0.800019,31.423077,4,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407
4,1,22.953205,11.076923,2.923077,4.846154,1.346154,3.961538,1.192308,1.461538,1.076923,...,0.333333,24.307692,5,0.936315,0.970874,0.966889,76.653846,78.346154,0.185185,0.407407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24503,1,12.859524,8.428571,2.142857,3.857143,1.071429,2.071429,0.928571,1.285714,0.714286,...,0.568182,39.866667,6,1.070937,1.020805,1.010547,80.800000,86.214286,0.533333,0.500000
24504,1,24.921429,7.857143,3.142857,4.785714,0.000000,0.142857,1.571429,2.214286,1.357143,...,0.726744,17.214286,7,1.070937,1.020805,1.010547,80.800000,86.214286,0.533333,0.500000
24505,1,16.352381,5.714286,0.857143,1.714286,1.071429,2.000000,0.785714,1.142857,0.142857,...,1.102941,39.866667,8,1.070937,1.020805,1.010547,80.800000,86.214286,0.533333,0.500000
24506,1,10.650000,2.500000,0.714286,1.285714,0.357143,0.714286,0.000000,0.000000,0.571429,...,1.125000,26.642857,9,1.070937,1.020805,1.010547,80.800000,86.214286,0.533333,0.500000


In [9]:
y

0         7
1        13
2        17
3         5
4        12
         ..
24503     2
24504    16
24505    16
24506     0
24507     0
Name: points, Length: 24508, dtype: int64

#### Podela podataka na trening, validacioni i test skup podataka

In [10]:
test_indices=final_player_stats.index[final_player_stats['points_pros'].notnull()]
X_train_validation,X_test,y_train_validation,y_test=train_test_split(X,y,train_size=0.8,random_state=42)
test_indices_in_train=set(test_indices).intersection(set(X_train_validation.index))
X_test = pd.concat([X_test, X_train_validation.loc[list(test_indices_in_train)]])
y_test = pd.concat([y_test, y_train_validation.loc[list(test_indices_in_train)]])
X_train_validation.drop(index=test_indices_in_train,inplace=True)
y_train_validation.drop(index=test_indices_in_train,inplace=True)

In [11]:
print("X_train_validation shape:",X_train_validation.shape)
print("y_train_validation shape:",y_train_validation.shape)
print("X_test shape:",X_test.shape)
print("y_test shape:",y_test.shape)

X_train_validation shape: (17794, 106)
y_train_validation shape: (17794,)
X_test shape: (6714, 106)
y_test shape: (6714,)


#### Standardizacija

In [12]:
scaler=StandardScaler()
scaler.fit(X_train_validation)
X_train_validation=scaler.transform(X_train_validation)
X_test=scaler.transform(X_test)

#### Testiranje modela AdaBoostRegressor

In [13]:
base_model=LinearRegression()
grid_model=AdaBoostRegressor(estimator=base_model)

params = {
    'n_estimators':[50,100],
    'learning_rate':[0.01,0.1],
}

grid = GridSearchCV(grid_model, param_grid=params, scoring='neg_mean_absolute_error', cv=5)
grid.fit(X_train_validation, y_train_validation)


GridSearchCV(cv=5, estimator=AdaBoostRegressor(estimator=LinearRegression()),
             param_grid={'learning_rate': [0.01, 0.1],
                         'n_estimators': [50, 100]},
             scoring='neg_mean_absolute_error')

In [14]:
print(f"Model: {grid_model}")
print(f"Best parameters:\n\tn_estimators: {grid.best_params_['n_estimators']} \n\tlearning_rate: {grid.best_params_['learning_rate']} ")

Model: AdaBoostRegressor(estimator=LinearRegression())
Best parameters:
	n_estimators: 50 
	learning_rate: 0.01 


In [15]:
y_test_pred=grid.best_estimator_.predict(X_test)
y_train_pred=grid.best_estimator_.predict(X_train_validation)


print('Test results:')
print('r2 score: ',r2_score(y_test, y_test_pred))
print('Mean absolute error: ', mean_absolute_error(y_test, y_test_pred))

print('\n\nTrain results:')
print('r2 score: ', r2_score(y_train_validation, y_train_pred))
print('Mean absolute error: ', mean_absolute_error(y_train_validation, y_train_pred))


Test results:
r2 score:  0.2874040031536891
Mean absolute error:  4.14973812988614


Train results:
r2 score:  0.3149905476330972
Mean absolute error:  3.9745281171381195


In [16]:
def proveri_vrednosti(row):
    if row['predicted'] > row['points_pros']:
        return '+'
    else:
        return '-'


y_test=pd.DataFrame(y_test)
y_test['predicted']=y_test_pred

predicted_vs_actual_pts_scored=y_test.rename({'points':'actual_points'})
predicted_vs_actual_pts_scored
predicted_player_points=final_player_stats[['player','points_pros','date']].merge(predicted_vs_actual_pts_scored,left_index=True,right_index=True)
predicted_player_points.dropna(inplace=True)
predicted_player_points['tip']=predicted_player_points.apply(proveri_vrednosti,axis=1)
predicted_player_points

,player,points_pros,date,points,predicted,tip
0,Timothe Luwawu-Cabarrot,10.5,2024-03-01,7,13.118313,+
2,Joffrey Lauvergne,11.5,2024-03-01,17,10.919635,-
3,Paris Lee,11.5,2024-03-01,5,12.228939,+
4,Mike Scott,10.5,2024-03-01,12,10.260699,-
8,Shavon Shields,16.5,2024-03-01,9,14.855084,-
...,...,...,...,...,...,...
8254,Thomas Walkup,8.5,2022-11-24,4,8.428198,-
8255,Kostas Papanikolaou,8.5,2022-11-24,19,7.624295,-
8257,Alec Peters,13.5,2022-11-24,0,6.312059,-
8258,Isaiah Canaan,11.5,2022-11-24,3,6.823476,-


#### Provera tačnosti modela u predviđanju broja poena igrača u odnosu na granicu bukmejkera

In [17]:
def proveri_pogodak(row):
    if row['points']>row['points_pros'] and row['predicted']>row['points_pros'] or row['points']<row['points_pros'] and row['predicted']<row['points_pros']:
        return 1
    else:
        return 0

predicted_player_points['pogodak'] = predicted_player_points.apply(proveri_pogodak, axis=1)

tacna_predvidjanja=predicted_player_points[predicted_player_points['pogodak']==1].count().unique()[0]
netacna_predvidjanja=predicted_player_points[predicted_player_points['pogodak']==0].count().unique()[0]

print("Broj tacnih predvidjanja:",tacna_predvidjanja)
print("Broj netacnih predvidjanja:",netacna_predvidjanja)

print("")

print(f"Procenat uspesnosti predvidjanja: {np.round(tacna_predvidjanja/(tacna_predvidjanja+netacna_predvidjanja)*100,2)}%")

Broj tacnih predvidjanja: 1293
Broj netacnih predvidjanja: 943

Procenat uspesnosti predvidjanja: 57.83%
